In [11]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
import os
import sys
import logging
from openpyxl import load_workbook
from datetime import datetime, date
import pandas as pd
import threading
from pathos.multiprocessing import ProcessingPool as Pool
from concurrent.futures import ProcessPoolExecutor, as_completed
import concurrent.futures
from trade.assets.Stock import Stock
from trade.helpers.helper import generate_option_tick_new
from trade.assets.rates import get_risk_free_rate_helper
from trade.helpers.helper import IV_handler, time_distance_helper, binomial_implied_vol, wait_for_response
from trade.helpers.helper import extract_numeric_value, change_to_last_busday
from trade.helpers.Logging import setup_logger
from trade.assets.Calculate import Calculate
from trade.helpers.Context import Context
from dbase.DataAPI.ThetaData import retrieve_ohlc, retrieve_quote_rt, retrieve_eod_ohlc, resample, retrieve_quote
from dbase.DataAPI.Organizers import generate_optionData_to_save, Calc_Risks
from dbase.database.SQLHelpers import store_SQL_data_Insert_Ignore, query_database, dynamic_batch_update
from trade.helpers.decorators import log_error, log_error_with_stack, log_time
from trade.helpers.types import OptionModelAttributes
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BDay
from dbase.database.SQLHelpers import DatabaseAdapter
from abc import ABC, abstractmethod
logger = setup_logger('test_datamanager')

Using Proxy URL: http://18.232.166.224:5500/thetadata


In [12]:
__all__ = [
    'DataManagerBase',
    'OptionDataManager',
    'SpotDataManager',
    'VolDataManager',
    'GreeksDataManager',
    'AttributionDataManager',
]

In [13]:

## Format of tables is: database_name.table_name
TABLES = {
    'eod':{
        'attribution': 'securities_master.attribution_eod',
        'spot': 'securities_master.temp_options_eod',
        'vol': 'securities_master.temp_options_eod',
        'greeks': 'securities_master.temp_options_eod',
        'chain': 'vol_surface.option_chain'
    },
    'intra':{
        'attribution': 'securities_master.attribution_intra',
        'spot': 'securities_master.temp_options_intra',
        'vol': 'securities_master.temp_options_intra',
        'greeks': 'securities_master.temp_options_intra',
    }
}


In [144]:
class OptionQueryRequestParameter:
    def __init__(self, table_name, db_name, start_date=None, end_date=None, ticker=None, exp=None, strike=None):
        self.db_name = db_name
        self.table_name = table_name
        self.start_date = start_date
        self.end_date = end_date
        self.ticker = ticker
        self.exp = exp
        self.strike  = strike
        self.opttick= None
        self.query = None
        self.y = None
        self.vol = None
        self.spot = None
        self.interval = None
        self.type_ = None

In [153]:
class OptionDataManager:
    @log_time(logger)
    def __init__(self,
                symbol: str = None,
                exp: str | datetime | date = None,
                right: str = None,
                strike: float = None,
                default_fill: str = 'midpoint',
                opttick: str = None,
                **kwargs) -> None:
        """
        Returns an object for querying data

        Params:
        symbol: Underlier symbol
        exp: expiration
        right: Put(P) or Call (C)
        strike: Option Strike
        default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
        opttick: Option ticker, if provided, will ignore symbol, exp, right, strike and be initialized with the string
        """

        if opttick is not None:
            assert isinstance(opttick, str), f"opttick has to be type str, recieved {type(opttick)}"
            option_meta = parse_option_tick(opttick)
            self.symbol = option_meta['ticker']
            self.exp = option_meta['exp_date']
            self.right = option_meta['put_call']
            self.strike = option_meta['strike']
            self.opttick = opttick

        else:
            assert isinstance(strike, float), f"Strike has to be type float, recieved {type(strike)}"
            if default_fill not in ['midpoint', 'weighted_midpoint', None]:
                raise ValueError("Expected default_fill to be one of: 'midpoint', 'weighted_midpoint', None ")
            
            assert all([symbol, exp, right, strike]), "symbol, exp, right, strike are required"
            self.exp = exp
            self.symbol = symbol
            self.right = right.upper()
            self.strike = strike
            self.opttick = generate_option_tick_new(symbol, right, exp, strike)

        self.default_fill = default_fill
        self.db = DatabaseAdapter()
        self.Stock = Stock(self.symbol, run_chain = False)
        ## Prefer to use dicts to avoid having too many attributes
        self._intra = {}
        self._eod = {}
    
    @property
    def eod(self):
        """
        Returns the end of day data
        """
        class EODData(dict):
            def __init__(inner, parent):
                inner.parent = parent
                super().__init__()

            def __getitem__(inner, key): ## Custom getter for EOD Dict. To initialize the data, if not already done
                if key not in inner.parent._eod:
                    if key not in ['s0_close', 's0_chain', 'r', 'y']:
                        raise KeyError(f"{key} not in eod data, expected one of: ['s0_close', 's0_chain', 'r', 'y']")
                    inner.parent._eod[key] = inner.parent._lazy_load(key)
                return inner.parent._eod[key]
            
            def __contains__(innner, key):
                return key in inner.parent._eod
            
            def __repr__(inner):
                return inner.parent._eod.__repr__()
            
            def __len__(inner):
                return len(inner.parent._eod)
            
            def keys(inner):
                return inner.parent._eod.keys()
        return EODData(self)
    
    @property
    def intra(self):
        """
        Returns the end of day data
        """
        class IntraData(dict):
            def __init__(inner, parent):
                inner.parent = parent
                super().__init__()

            def __getitem__(inner, key): ## Custom getter for EOD Dict. To initialize the data, if not already done
                if key not in inner.parent._intra:
                    if key not in ['s0_close', 's0_chain', 'r', 'y']:
                        raise KeyError(f"{key} not in intra data, expected one of: ['s0_close', 's0_chain', 'r', 'y']")
                    inner.parent._intra[key] = inner.parent._lazy_load(key, ts_timewidth = '5', ts_timeframe = 'minute')
                return inner.parent._intra[key]
            
            def __contains__(innner, key):
                return key in inner.parent._intra
            
            def __repr__(inner):
                return inner.parent._intra.__repr__()
            
            def __len__(inner):
                return len(inner.parent._intra)
            
            def keys(inner):
                return inner.parent._intra.keys()
        return IntraData(self)

    def _lazy_load(self, load_name, **kwargs):
        ## Utilizing the lazy load function to load data on demand, and speed up initialization
        if load_name == 's0_close':

            ## Will use Kwargs to move between intra and EOD.
            return self.Stock.spot(ts = True,
                                          ts_start = pd.to_datetime(self.exp) - relativedelta(years=2),
                                          ts_end =pd.to_datetime(self.exp) + relativedelta(years=2),
                                          **kwargs)
        elif load_name == 's0_chain':
            return self.Stock.spot(ts = True,
                                            ts_start = pd.to_datetime(self.exp) - relativedelta(years=2),
                                            ts_end =pd.to_datetime(self.exp) + relativedelta(years=2),
                                            spot_type='chain_price',
                                            **kwargs)
            
        elif load_name == 'r':
            return get_risk_free_rate_helper()

        elif load_name == 'y':
            return self.Stock.div_yield_history()

    def get_timeseries(self, 
                       start: str | datetime, 
                       end: str | datetime,
                       interval: str = '1d',
                       type_: str = 'spot',
                       model: str = 'bs') -> pd.DataFrame:
        
        
        ## Organize inputs
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        ivl_str, ivl_int = extract_numeric_value(interval)
        greek_names = ['vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greek', 'greeks']


        ## Assert inputs
        if type_ not in ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution', 'scenario']:
            raise ValueError("Expected type_ to be one of: ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution', 'scenario']")
        if model not in ['bs', 'bt', 'mc', 'bsm']: ## Only Black Scholes, binomial tree, monte carlo
            raise ValueError("Expected model to be one of: ['bs', 'bt', 'mc', 'bsm']")
        
        if ivl_str.lower() not in ['d', 'w','q','y', 'h'] and ivl_str != 'M': ## Want to avoid minute data
            raise ValueError("Expected interval to be one of: ['d', 'w','q','y' 'M']")
        
        if ivl_str == 'm': ## Minute data not available
            raise AttributeError("Minute data currently unavailable, please go higher")
        

        ## Determine aggregation
        if ivl_str == 'h':
            agg = 'intra'
        else:
            agg = 'eod'
        
        ## Table to query, picking based on interval & type
        if type_ in greek_names:
            database, table = TABLES[agg]['greeks'].split('.')
        else:
            database, table = TABLES[agg][type_].split('.')
        print(database, table)

        data_request = OptionQueryRequestParameter(table_name=table, 
                                                   db_name=database, 
                                                   start_date=start, 
                                                   end_date=end, 
                                                   ticker=self.symbol, 
                                                   exp=self.exp, 
                                                   strike=self.strike)
        data_request.opttick = self.opttick
        data_request.interval= interval
        data_request.type_ = type_

        self.__init_query(data_request=data_request)
        return data_request
    
    def get_at_time(self, 
                   date: str | datetime, 
                   end: str | datetime,
                   type_: str = 'spot',
                   model: str = 'bs') -> pd.DataFrame:
        """
        Get data at a specific time
        """
        pass

    def __init_query(self, **kwargs):
        data_request = kwargs.get('data_request')
        query = f"""SELECT *
        FROM {data_request.db_name}.{data_request.table_name}
        WHERE OPTIONTICK = '{data_request.opttick}'
        """
        database_data = self.db.query_database(data_request.db_name, data_request.table_name, query)
        data_request.query = query
        data_request.database_data = database_data
        return database_data

    def __verify_data_completeness(self, data: pd.DataFrame, interval_type: str) -> 'Something':
        """
        Verify that the data is complete
        """
        pass



class SpotDataManager:
    pass

class VolDataManager:
    pass

class GreeksDataManager:
    pass

class AttributionDataManager:
    pass

class ChainDataManager:
    pass

class ScenarioDataManager:
    pass

In [147]:
OptionDataManager?

Init signature:
OptionDataManager(
    symbol: str = None,
    exp: str | datetime.datetime | datetime.date = None,
    right: str = None,
    strike: float = None,
    default_fill: str = 'midpoint',
    opttick: str = None,
    **kwargs,
) -> None
Docstring:      <no docstring>
Init docstring:
Returns an object for querying data

Params:
symbol: Underlier symbol
exp: expiration
right: Put(P) or Call (C)
strike: Option Strike
default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
opttick: Option ticker, if provided, will ignore symbol, exp, right, strike and be initialized with the string
Type:           type
Subclasses:     

In [154]:
manager = OptionDataManager('AAPL', '2025-04-11', 'C', 145.0)

In [157]:
request = manager.get_timeseries(
    start='2024-01-01',
    end='2024-12-31',
    interval='1d',
    type_='spot',
    model='bs'
)
request.database_data

securities_master temp_options_eod


,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Strike,...,weighted_midpoint_Vega,weighted_midpoint_Theta,weighted_midpoint_Rho,weighted_midpoint_Vanna,weighted_midpoint_Volga,weighted_midpoint_Dollar_Delta,OpenInterest,bid_IV,ask_IV,last_updated



Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-04-04 17:04:21 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs


In [9]:
db = DatabaseAdapter()
query = """
SELECT * FROM SECURITIES_MASTER.TEMP_OPTIONS_EOD
LIMIT 10000"""

data = db.query_database('securities_master', 'temp_options_eod', query)
data

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Strike,...,weighted_midpoint_Vega,weighted_midpoint_Theta,weighted_midpoint_Rho,weighted_midpoint_Vanna,weighted_midpoint_Volga,weighted_midpoint_Dollar_Delta,OpenInterest,bid_IV,ask_IV,last_updated
0,7.20,7.35,6.80,7.35,42,326,6.95,128,7.50,120.0,...,0.336261,-0.020056,0.239003,0.668221,24.872587,31.928200,4477,0.000000,0.000000,2025-02-27 06:10:14
1,5.65,5.65,5.15,5.49,16,250,5.35,150,5.90,130.0,...,0.308340,-0.018748,0.198361,0.657378,35.021027,26.157776,2685,0.000000,0.000000,2025-02-27 06:10:12
2,7.40,8.09,7.40,8.09,102,401,7.70,163,8.15,120.0,...,0.347200,-0.021141,0.252368,0.655590,21.764441,34.116137,4481,0.000000,0.000000,2025-02-27 06:10:14
3,5.55,6.10,5.55,6.10,43,1,5.65,225,6.20,130.0,...,0.321696,-0.019993,0.211612,0.655330,32.205635,28.238159,2698,0.000000,0.000000,2025-02-27 06:10:12
4,2.78,3.07,2.78,3.05,58,212,3.05,64,3.20,195.0,...,0.338953,-0.014042,0.189505,0.699666,95.010554,23.046435,1990,0.291244,0.295632,2025-02-27 06:17:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,49.45,49.45,49.45,49.45,1,42,50.05,10,51.95,560.0,...,1.876698,-0.105185,2.246148,-0.293272,41.037687,340.679848,320,0.204840,0.214910,2025-03-19 07:27:02
9996,4.95,5.05,4.77,4.89,174,22,4.80,26,5.05,40.0,...,0.131983,-0.009912,0.124634,0.285984,-1.329609,21.040553,13614,0.373382,0.392215,2025-02-24 00:34:39
9997,3.00,3.05,2.94,2.94,266,69,2.86,113,3.15,45.0,...,0.129642,-0.009245,0.096285,0.603400,4.789204,15.481647,11086,0.354284,0.376509,2025-02-24 00:34:39
9998,0.00,0.00,0.00,0.00,0,18,142.85,18,145.05,165.0,...,0.086958,-0.041546,0.546255,-2.991623,1114.604880,299.017535,3583,0.000000,0.656274,2025-04-03 00:37:34
